# Getting data ready for SNA in Gephi

In [1]:
import pandas as pd
from collections import defaultdict

# Load the data
data = pd.read_excel('Crypto_Fund_List.xlsx', header=1)

# Define the columns of interest
columns_to_select = [
    'Firm Name',
    'Investments 1',
    'Investments 2',
    'Investments 3',
    'Investments 4',
    'Investments 5'
]

# Select the data
base_df = data[columns_to_select].fillna(0)

# Filter firms with at least 5 non-zero investments
base_df['Non_zero_investments'] = base_df[columns_to_select[1:]].astype(bool).sum(axis=1)
filtered_base_df = base_df[base_df['Non_zero_investments'] >= 5]

# Create unique labels for nodes from the filtered dataframe
unique_labels = pd.unique(filtered_base_df[columns_to_select].values.ravel())

# Remove the zero label if it's there
unique_labels = unique_labels[unique_labels != 0]

# Create the nodes DataFrame
nodes_df = pd.DataFrame({'id': range(1, len(unique_labels) + 1), 'Label': unique_labels})

# Initialize the degree dictionary
degree_dict = defaultdict(int)

# Calculate degree for each node
for _, row in filtered_base_df.iterrows():
    for investment in row[columns_to_select[1:]]:
        if investment != 0:
            degree_dict[investment] += 1

# Add a 'Type' column to nodes_df for distinguishing between firms and coins
nodes_df['Type'] = nodes_df['Label'].apply(lambda x: 'Coin' if degree_dict[x] > 0 else 'Firm')

# Map labels to IDs
label_to_id = dict(zip(nodes_df['Label'], nodes_df['id']))

# Initialize the edges list
edges = []

# Create edges
for _, row in filtered_base_df.iterrows():
    firm_name = row['Firm Name']
    for investment in row[columns_to_select[1:]]:
        if investment != 0:
            source_id = label_to_id[firm_name]
            target_id = label_to_id[investment]
            edges.append({'Source': source_id, 'Target': target_id, 'Type': 'Directed'})

# Create the edges DataFrame
edges_df = pd.DataFrame(edges)

# Add a 'Type_Num' column to nodes_df for sizing in Gephi
nodes_df['Type_Num'] = nodes_df['Type'].apply(lambda x: 1 if x == 'Firm' else 2)

# Output the shape of the resulting DataFrames
print('Nodes shape:', nodes_df.shape)
print('Edges shape:', edges_df.shape)


Nodes shape: (128, 4)
Edges shape: (225, 3)


In [53]:
# Export the nodes and edges to CSV files for Gephi
nodes_df.to_csv('nodes_filtered.csv', index=False)
edges_df.to_csv('edges_filtered.csv', index=False)
print('Exported nodes and edges to CSV')

Exported nodes and edges to CSV


In [3]:
filtered_base_df.head()

,Firm Name,Investments 1,Investments 2,Investments 3,Investments 4,Investments 5,Non_zero_investments
9,All in One Coin,Bitcoin,Ethereum,Ripple,Bitcoin Cash,Litecoin,5
21,Andreessen Horowitz,dyDx,Ripple,Coinbase,Axoni,Basecoin,5
29,Authorito,Zcash,Monero,Ethereum,Ripple,Bitcoin,5
47,Bitwise Asset Management,Bitcoin,Ethereum,Ripple,Bitcoin Cash,Litecoin,5
53,Block Bits Capital,ICOs,Bitcoin,Ethereum,Ripple,Litecoin,5


In [4]:
nodes_df.head()

,id,Label,Type,Type_Num
0,1,All in One Coin,Firm,1
1,2,Bitcoin,Coin,2
2,3,Ethereum,Coin,2
3,4,Ripple,Coin,2
4,5,Bitcoin Cash,Coin,2


In [5]:
edges_df.head()

,Source,Target,Type
0,1,2,Directed
1,1,3,Directed
2,1,4,Directed
3,1,5,Directed
4,1,6,Directed
